## Visualization of CNN: Grad-CAM
* **Objective**: Convolutional Neural Networks are widely used on computer vision. It is powerful for processing grid-like data. However we hardly know how and why it works, due to the lack of decomposability into individually intuitive components. In this assignment, we use Grad-CAM, which highlights the regions of the input image that were important for the neural network prediction.

* **To be submitted by next session**: this notebook, **cleaned** (i.e. without results, for file size reasons: `menu > kernel > restart and clean`), in a state ready to be executed (if one just presses 'Enter' till the end, one should obtain all the results for all images) with a few comments at the end. No additional report, just the notebook!

* NB: if `PIL` is not installed, try `conda install pillow`.


In [ ]:
import torch
import torch.nn as nn
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image

%matplotlib inline

### Download the Model
We provide you a pretrained model `VGG-16` for `ImageNet` classification dataset.
* **ImageNet**: A large dataset of photographs with 1 000 classes.
* **VGG-16**: A deep architecture for image classification.

![vgg_16.png](https://www.researchgate.net/profile/Bibo_Shi/publication/323440752/figure/fig1/AS:739814685032448@1553396974148/The-architecture-of-VGG-16-model-To-represent-different-depth-levels-convolutional.jpg)

In [ ]:
# The downloading process may take a few minutes. 
vgg_model = models.vgg16(pretrained=True)# return the vgg-16 model pretrained on ImageNet dataset.

### Input Images
We provide you 20 images from ImageNet (download link on the webpage of the course; notice that the images should be placed in a **sub**-directory of the path indicated below).<br>
In order to use the pretrained model vgg-16, the input image should be normalized using `mean = [0.485, 0.456, 0.406]`, and `std = [0.229, 0.224, 0.225]`, and be resized as `(224, 224)`.

In [ ]:
# Define preprocessing function of the input images
def preprocess_image(dir_path):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    dataset = datasets.ImageFolder(dir_path, transforms.Compose([
            transforms.Resize(256), 
            transforms.CenterCrop(224), # resize the image to 224x224
            transforms.ToTensor(), # convert numpy.array to tensor
            normalize])) #normalize the tensor

    return (dataset)

In [ ]:
# The images should be in a *sub*-folder of "data/" (ex: data/TP2_images/images.jpg) and *not* directly in "data/"!
# otherwise the function won't find them
dir_path = "data/" 
dataset = preprocess_image(dir_path)

In [ ]:
# show the orignal image 
index = 5
input_image = Image.open(dataset.imgs[index][0]).convert('RGB')
plt.imshow(input_image);

Predict the label of the input image, and print the top-3 possible classes

In [ ]:
output = vgg_model(dataset[index][0].view(1, 3, 224, 224))

values, indices = torch.topk(output, 3)
print("Top 3-classes:", indices[0].numpy())
print("Raw class scores:", values[0].detach().numpy())

### Grad-CAM 
* **Overview:** Given an image, and a category (‘tiger cat’) as input, we forward-propagate the image through the model to obtain the `raw class scores` before softmax. The gradients are set to zero for all classes except the desired class (tiger cat), which is set to 1. This signal is then backpropagated to the `rectified convolutional feature map` of interest, where we can compute the coarse Grad-CAM localization (blue heatmap).


* **To Do**: Define your own function Grad_CAM to achieve the visualization of the given images. For each image, choose the top-3 possible labels as the desired classes. Compare the heatmaps of the three classes, and conclude. 


* **Hints**: 
 + We need to record the output and grad_output of the feature maps to achieve Grad-CAM. In pytorch, the function `Hook` is defined for this purpose. Read the tutorial of [hook](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html#forward-and-backward-function-hooks) carefully. 
 + The pretrained model vgg-16 doesn't have an activation function after its last layer, the output is indeed the `raw class scores`, you can use them directly. Run `print(vgg_model)` to get more information on VGG model.
 + The size of feature maps is 14x14, so your heatmap will have the same size. You need to project the heatmap to the resized image (224x224, not the original one, before the normalization) to have a better observation. The function [`torch.nn.functional.interpolate`](https://pytorch.org/docs/stable/nn.functional.html?highlight=interpolate#torch.nn.functional.interpolate) may help.  
 + Here is the link of the paper [Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization](https://arxiv.org/pdf/1610.02391.pdf)

![Grad_CAM](https://upload-images.jianshu.io/upload_images/415974-0147c44dcfb8cc1c.jpg)

In [ ]:
print(vgg_model)

The convolutional layer where we want to extract the gradients from is the 29th one

Its name is `features.29` and it is in the 31st position of the list `modules` built here

In [ ]:
modules = []
for name, module in vgg_model.named_modules():
    modules.append(module)

We first define the hooks that will store the values of the :
- Activations : values that the last rectified convolutional feature map outputs during the forward pass
- gradients : values that the last rectified convolutional feature map outputs during the backward pass

In [ ]:
activation = {}
def forward_hook(m, input, output):
     activation[m] = output
gradient = {}
def backward_hook(m, grad_in, grad_out):
     gradient[m] = grad_out[0]
        
input_ = dataset[index][0].view(1, 3, 224, 224)
modules[31].register_forward_hook(forward_hook)
modules[31].register_backward_hook(backward_hook)

By executing a prediction, a forward pass will be executed and thus store the values in the dictionnary `activation` as we can see below

In [ ]:
vgg_model.eval() # Ensure that the dropout layers are not used while predicting
prediction = vgg_model(input_)

In [ ]:
activation

We take the highest score outputed by the last layer (unnormalized) and then we back propagate it (partial derivatives). Thus the backward hook we placed before will retain the value of the gradient of this score wrt to the activations. 

In [ ]:
prediction[:,prediction.argmax(dim=1)].backward()

In [ ]:
gradient

Notation <==> variable  

$y^c$ <==> Highest output score

$A_{ij}^k$ <==> The activation of the pixel in the position $(i,j)$ of the $k$-th channel

- The values contained in the `activation` dictionnary are the $A_{ij}^k$
- The values contained in the `gradient` dictionnary are the $\frac{\partial y^c}{\partial A_{ij}^k}$

Equation (1) in the paper say we have to perform global average pooling on the gradient values : 
$$\alpha_k^c = \frac{1}{Z}\sum_{ij} \frac{\partial y^c}{\partial A_{ij}^k} $$  where $Z$ is a normalisation factor

We thus obtain $k$ values one for each channel

In [ ]:
pooled_gradient = torch.mean(gradient[modules[31]], dim=[0, 2, 3]) # equation number 1 in the paper

After that we construct a linear combination of those coefficients $\alpha_k^c$ and $A_{ij}^k$ sum them along the $k$ dimensions and only keep the positive factors of this sum. This is equation (2) in the paper.

$$ L_{ij}^c =  ReLU (\sum_k \alpha_k^c A_{ij}^k ) $$

In [ ]:
linear_combination = torch.Tensor(activation[modules[31]][0])
for chann in range(512) :
    linear_combination[chann, : , :] *=  pooled_gradient[chann] # equation number 2 in the paper
heatmap = nn.ReLU()(torch.sum(linear_combination, dim=0, keepdim=True)) # equation number 2 in the paper

Visualizing the pixels obtained 

In [ ]:
plt.matshow(heatmap.squeeze().detach().numpy());

Now we just have to interpolate this $14x14$ picture with the original one after normalisation ($224x224$) in order to superimpose the two

In [ ]:
heat = torch.nn.functional.interpolate(heatmap.unsqueeze(0),
                                       (224,224), 
                                       mode="bilinear", 
                                       align_corners=False
                                       )

And here is the final result

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(input_.squeeze(0).permute(1,2,0).detach().numpy())
plt.imshow(heat.squeeze().detach().numpy(), alpha=0.85)
plt.show()

We see that some area on the picture are highlighted ( in yellow/green) indicating the areas and pixels that influenced the most the output score of the model.

***

We wrap all this in this function : We give it an image and an input, and outputs the most influencing area of the top k prediction scores

In [ ]:
def GRAD_Cam(input_, k):
    plt.figure(figsize=(15,10))
    prediction = vgg_model(input_)
    values, indices = torch.topk(prediction, 3)
    
    for i in range(k) :
        
        activation = {}
        def forward_hook(m, input, output):
             activation[m] = output
        gradient = {}
        def backward_hook(m, grad_in, grad_out):
             gradient[m] = grad_out[0]

        vgg_model.eval()
        modules[31].register_forward_hook(forward_hook)
        modules[31].register_backward_hook(backward_hook)
        prediction = vgg_model(input_)
        
        prediction[:,prediction.argsort(descending=True)[:,i]].backward()

        title = str(indices[0][i])
        
        pooled_gradient = torch.mean(gradient[modules[31]], dim=[0, 2, 3]) # equation number 1 in the paper

        linear_combination = torch.Tensor(activation[modules[31]][0])
        
        for chann in range(512) :
            linear_combination[chann, : , :] *=  pooled_gradient[chann] # equation number 2 in the paper
            
        heatmap = nn.ReLU()(torch.sum(linear_combination, dim=0, keepdim=True)) # equation number 2 in the paper
        
        heat = torch.nn.functional.interpolate(heatmap.unsqueeze(0),
                                           (224,224), 
                                           mode="bilinear", 
                                           align_corners=False
                                           )
        plt.subplot(1,k,i+1)
#         plt.suptitle(title)
        plt.imshow(input_.squeeze(0).permute(1,2,0).detach().numpy())
        plt.imshow(heat.squeeze().detach().numpy(), alpha=0.85)
    plt.show()

In [ ]:
index = 1
input_image = Image.open(dataset.imgs[index][0]).convert('RGB')
plt.imshow(input_image);
GRAD_Cam(dataset[index][0].view(1, 3, 224, 224), 3)

In [ ]:
index = 0
input_image = Image.open(dataset.imgs[index][0]).convert('RGB')
plt.imshow(input_image);
GRAD_Cam(dataset[index][0].view(1, 3, 224, 224), 3)

In [ ]:
index = 19
input_image = Image.open(dataset.imgs[index][0]).convert('RGB')
plt.imshow(input_image);
GRAD_Cam(dataset[index][0].view(1, 3, 224, 224), 3)

In [ ]:
index = 15
input_image = Image.open(dataset.imgs[index][0]).convert('RGB')
plt.imshow(input_image);
GRAD_Cam(dataset[index][0].view(1, 3, 224, 224), 3)

We see that sometimes, the model focuses on the same areas of the image to output the scores of the classes, but those scores are nevertheless reflecting its confidence on the class of interest ( the class we are backpropagating its score).

We also see that the network is not totally independant from the background since it gives it a little bit of attention ( maybe it helps him being confident on the prediction he gives - we are less likely to see a cow in a bedroom- so maybe that is only a toy ).